In [1]:
from itertools import count

from print_better_dic import print_dict
from extract_artcls_data import current_artcls, artcl_data, artcl_content
from location_detector import detect_location


In [2]:
# Base url
news_url= "https://www.elnuevodia.com/noticias/seguridad"

In [3]:
data_handler= {}
stats_handler= {}

articles= current_artcls(news_url)

for _ in zip(count(start= 1, step= 1), articles):
    data_handler[str(_[0])]= artcl_data(article= _[1])
    data_handler[str(_[0])].update(artcl_content(article_data= data_handler[str(_[0])]))
    data_handler[str(_[0])].update(detect_location(article_data= data_handler[str(_[0])]))

In [4]:
print(print_dict(data_handler))

1: 
    headline: Piloto muere luego de que su avioneta se estrellara en área residencial en Aguadilla
    url: https://www.elnuevodia.com/noticias/seguridad/notas/piloto-muere-luego-de-que-su-avioneta-se-estrellara-en-area-residencial-en-aguadilla/
    author: Por Edlyn M. Vega Rodríguez y Alex Figueroa Cancel
    headline_sub: La aeronave, que era manejada por Wilfredo Juarbe Saldaña, de 47 años, había estado en reparaciones después de haber tenido desperfectos mecánicos
    date: 11 de julio de 2024
    time: 10:15 AMActualizado el 11 de julio de 2024
    content: Aguadilla - Eran las 9:34 de la mañana, apenas comenzaba el día, cuando un estruendo trastocó la tranquilidad de los vecinos del barrio Maleza Alta, en Aguadilla, y avisó la inesperada tragedia de que una avioneta se había estrellado en sus patios. El humo que provocó la catástrofe era “negro y colora’o, parecía el infierno”, contó la residente Gigi Lozano. Tras el incidente, muy asustada, salió de su casa, se armó de una 

In [5]:
# # Import module 
# import mysql.connector

# # Make the connection to databese
# connection= mysql.connector.connect(user= "root", password= "root", host= "mysql", port= "3306", database= "db")
# print("DB Connected!")

# # Create cursor obj
# cursor= connection.cursor(buffered= True)

In [6]:
# # Create variable with MySQL query to insert data into articles_data table
# data_query= """
# INSERT INTO articles_data (headline,
#                            url,
#                            author,
#                            date,
#                            time,
#                            location)
# VALUES (%(headline)s, %(url)s, %(author)s, %(date)s, %(time)s, %(pueblos)s) 
# """

# # Get the data from data_handler and format into dict so MySQL can understand it
# data= {"headline": data_handler["2"]["headline"],
#        "url": data_handler["2"]["url"],
#        "author": data_handler["2"]["author"],
#        "date": data_handler["2"]["date"],
#        "time": data_handler["2"]["time"],
#        "pueblos": data_handler["2"]["pueblos"][0]} # This must be rewritten in a way that MySQL can store lists in the field,
#                                                    # right now I just picked the first item from the returned list

# # Execute the query with data provided
# cursor.execute(data_query, data)

# # Make sure data is committed to the database
# # Commented for testing but must be done when not testing to make sure data is saved in the database table
# # connection.commit()

In [7]:
# # Retrieve all data from the table
# cursor.execute("SELECT * FROM articles_data")

# # Print each table row
# for row in cursor:
#     print(row)